In [ ]:
import pandas as pd
import numpy as np


In [ ]:
import spacy
from spacy.lang.en import English
from spacy.lang.es import Spanish
parser = English()
parser1 = Spanish()

In [ ]:
nlp = spacy.load('en_core_web_sm',disable='NER')
#nlp = spacy.load('es_core_web_sm')

In [ ]:
nlp.max_length = 6000000

In [ ]:
df = pd.read_csv('Tweets_with_senti.csv',sep='|' ,encoding= 'utf8')
df.columns

In [ ]:
df.to_csv('new_df.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
#df['text']= fillna(df['text'].mode(), inplace=True)
df['text'] = df['text'].fillna(df['text'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
#df.fullname.value_counts()

In [ ]:
df.search_keyword.value_counts()

In [ ]:
df.Sentiment.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Sentiment_le'] = le.fit_transform(df['Sentiment'])  

In [ ]:
df.Sentiment_le.value_counts()

In [ ]:
df['Sentiment_le'].head()

In [ ]:
df.head()


In [ ]:
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()

In [ ]:
# To build a list of stop words for filtering
stopwords = list(STOP_WORDS)
print(stopwords)

In [ ]:
import string
punctuations = string.punctuation
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(text)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
# Importing Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC


In [ ]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [ ]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = DecisionTreeClassifier()
classifier1= LinearSVC()
vectorizer

In [ ]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)
tfvectorizer

In [ ]:
# Splitting Data Set
from sklearn.model_selection import train_test_split

# Features and Labels
X = df.text
ylabels = df['Sentiment_le']

#X_train, X_test, y_train, y_test = train_test_split(X, ylabels,test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, ylabels,stratify=ylabels,test_size=0.3, random_state=42)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

# With Support Vector Machine


In [ ]:
# Create the  pipeline to clean, tokenize, vectorize, and classify using"Count Vectorizor" with SVM
pipe_countvect = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier1)])

In [ ]:
# Fit our data
pipe_countvect.fit(X_train,y_train)
# Predicting with a test dataset
sample_prediction = pipe_countvect.predict(X_test)

# Prediction Results
# 2 = Positive review
# 0 = Negative review
# 1 = Nuetral review
"""for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)"""

In [ ]:
# Accuracy
print("Accuracy: ",pipe_countvect.score(X_test,y_test))
print("Accuracy: ",pipe_countvect.score(X_test,sample_prediction))
# Accuracy
print("Accuracy: ",pipe_countvect.score(X_train,y_train))

# With Decision Tree Classifier

In [ ]:
# Create the  pipeline to clean, tokenize, vectorize, and classify using"Count Vectorizor" with DTC
pipe_countvect = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [ ]:
# Fit our data
pipe_countvect.fit(X_train,y_train)
# Predicting with a test dataset
y_pred = pipe_countvect.predict(X_test)

In [ ]:
# Accuracy
print("Accuracy: ",pipe_countvect.score(X_test,y_test))
print("Accuracy: ",pipe_countvect.score(X_test,y_pred))
# trained Accuracy
print("Accuracy: ",pipe_countvect.score(X_train,y_train))

In [ ]:
y_test.head()

In [ ]:
y_pred